#importing the libraries


In [3]:
import numpy as np
import pandas as pd

#Importing the dataset

In [4]:
rating_df = pd.read_csv('ratings.csv',usecols=['userId','movieId','rating'])
movie_df= pd.read_csv('movies.csv',usecols=['movieId','title'])

In [5]:
rating_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [6]:
movie_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


#Splitting the dataset

In [7]:
from sklearn.model_selection import train_test_split
train_rating_df,test_rating_df = train_test_split(rating_df,test_size=0.2,random_state=1)

#Create an user-item interaction pivot table

In [8]:
data_table = rating_df.pivot_table(index='movieId',columns='userId' , values='rating')
data_table =data_table.fillna(0)
train_table = train_rating_df.pivot_table(index='movieId',columns='userId' , values='rating')
train_table =train_table.fillna(0)

In [9]:
data_table.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
train_table.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,0.0,2.5,0.0,0.0,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


#Create KNN model

In [11]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(n_neighbors=20,algorithm='brute',metric='cosine')

#Train KNN on the training set

In [12]:
model.fit(train_table)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

#Reccomendation system

#Input the user to recommend movies to

In [13]:
selected_id= input(' Enter the userId of the user to recommend to : ')
selected_id = int(selected_id)

 Enter the userId of the user to recommend to : 100


#Getting the selected user's rated movies

In [14]:
selected_user= data_table[selected_id]
selected_user = selected_user.loc[selected_user!=0]
selected_user.head()

movieId
3     3.5
11    4.0
16    4.5
17    4.5
19    1.0
Name: 100, dtype: float64

#List of movies that rated by at least 50 users

In [15]:
stats= rating_df.groupby('movieId').agg({'rating':[np.size]})
stats.head(10 )
stats= stats[stats['rating']['size']>=50]

In [16]:
stats.head()

,rating
,size
movieId,
1,215
2,110
3,52
6,102
7,54


#Function to get recommendations and predict ratings that user would give to each movie

In [17]:
def get_recommendation():
  similar_candidates_rating= pd.Series(dtype='float64')
  similar_candidates_score= pd.Series(dtype='float64')
  for i in range(0,len(selected_user.index)):
    similar=model.kneighbors([data_table.loc[selected_user.index[i]]],return_distance=True)
    sim_score=similar[0]
    sim_id=similar[1]
    sim_id=np.array(list(map(lambda x: data_table.iloc[x].name,sim_id[0])))
    similar=pd.Series(data=sim_score[0],index=sim_id)
    similar=similar[similar!=0]
    similar=similar[similar.index.isin(stats.index)]
    similar_candidates_score=pd.concat([similar_candidates_score,similar])
    similar=similar.map(lambda x: x*selected_user.values[i])
    similar_candidates_rating = pd.concat([similar_candidates_rating,similar])
  return similar_candidates_rating,similar_candidates_score

In [18]:
similar_candidates_rating,similar_candidates_score= get_recommendation()

#Factoring the recommendation score to the scale of 1-5

In [19]:
similar_candidates_rating.sort_values(inplace=True,ascending=False)
similar_candidates_score.sort_values(inplace=True,ascending=False)
filtered_candidates_rating_sum= similar_candidates_rating.groupby(similar_candidates_rating.index).sum()
filtered_candidates_score_sum= similar_candidates_score.groupby(similar_candidates_score.index).sum()

In [20]:
similar_movies=filtered_candidates_rating_sum.index
pred_rating= pd.Series(dtype='float64',index=similar_movies)
for i in range(0,len(similar_movies)):
  pred_rating[similar_movies[i]]= filtered_candidates_rating_sum[similar_movies[i]]/filtered_candidates_score_sum[similar_movies[i]]

#Making Recommendations to the selected user

#The user's orignal highest rated films

In [21]:
selected_user.sort_values(inplace=True,ascending=False)
selected_user_df = pd.DataFrame(selected_user).reset_index()
selected_user_df.columns = ['movieId', 'predicted_rating']
selected_user_df = pd.merge(selected_user_df,movie_df)
selected_user_df.head(10)

,movieId,predicted_rating,title
0,1958,5.0,Terms of Endearment (1983)
1,2423,5.0,Christmas Vacation (National Lampoon's Christm...
2,5620,5.0,Sweet Home Alabama (2002)
3,1101,5.0,Top Gun (1986)
4,4041,5.0,"Officer and a Gentleman, An (1982)"
5,1307,4.5,When Harry Met Sally... (1989)
6,1912,4.5,Out of Sight (1998)
7,1777,4.5,"Wedding Singer, The (1998)"
8,1680,4.5,Sliding Doors (1998)
9,1678,4.5,"Joy Luck Club, The (1993)"


#Recommendations made by KNN

In [22]:
print("List of recommendations for user {}".format(selected_id))
pred_rating.sort_values(inplace=True,ascending=False)
pred_rating_df = pd.DataFrame(pred_rating).reset_index()
pred_rating_df.columns = ['movieId', 'predicted_rating']
final_pred_df = pd.merge(pred_rating_df,movie_df)
final_pred_df.head(10)

List of recommendations for user 100


,movieId,predicted_rating,title
0,2000,4.738382,Lethal Weapon (1987)
1,420,4.500000,Beverly Hills Cop III (1994)
2,6,4.500000,Heat (1995)
3,266,4.500000,Legends of the Fall (1994)
4,480,4.500000,Jurassic Park (1993)
5,552,4.500000,"Three Musketeers, The (1993)"
6,17,4.500000,Sense and Sensibility (1995)
7,904,4.500000,Rear Window (1954)
8,111,4.500000,Taxi Driver (1976)
9,110,4.500000,Braveheart (1995)
